In [1]:
%load_ext autoreload
%autoreload 2

from datasets.MSDWild import MSDWildChunks
from torch.utils.data import DataLoader
from pairs.config import S3_BUCKET_NAME, S3_VIDEO_DIR
import os
import torch
from torch.utils.data import DataLoader

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f541eb64390>>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


In [2]:
train_rttm_path = "few.train.rttm"
train_data_path = os.path.join("s3://", S3_BUCKET_NAME, S3_VIDEO_DIR)

In [3]:
dataset = MSDWildChunks(data_path=S3_VIDEO_DIR,
                        data_bucket=S3_BUCKET_NAME,
                        partition_path=train_rttm_path,
                        subset=0.03)

Loading Pair Metadata for Videos:  16%|█▌        | 395/2476 [00:33<02:54, 11.90it/s]


KeyboardInterrupt: 

In [ ]:
# s3fs -- 12/12 [03:04]
# boto3 - 12/12 [02:30]

In [ ]:
import torch
def collate_fn(batch):
    # Extract each feature: do the zip thing
    video_data, audio_data, is_speaking = list(zip(*batch))
    # Padding: NOTE: Not necessary
    # Stack:
    video_data = torch.stack(video_data)
    audio_data = torch.stack(audio_data)
    is_speaking = torch.tensor(is_speaking)
    # Return tuple((N, video_data, melspectrogram), (N, video_data, melspectrogram), (N, video_data, melspectrogram))
    # (N, C, H, W), (N, Bands, T) x3 (ask Prachi)
    batch_data = {
        "video_data": video_data,
        "audio_data": audio_data,
        "labels": is_speaking,
    }
    return batch_data

In [ ]:
from torch.utils.data import DataLoader
loader = DataLoader(
        dataset,
        batch_size=2,
        shuffle=True,
        collate_fn=collate_fn,
        num_workers=2,
        pin_memory=True,
    )

In [ ]:
for i in loader:
    print(i.keys())
    print(i["video_data"].shape)
    print(i["audio_data"].shape)
    print(i["labels"])
    break

In [ ]:
import torch
import boto3
import io

# Save the entire model
buffer = io.BytesIO()
torch.save(loader, buffer)
buffer.seek(0)

# Upload to S3
s3_client = boto3.client('s3')
s3_client.put_object(
    Bucket=S3_BUCKET_NAME, 
    Key="loaders/data_loader_01.pth", 
    Body=buffer.getvalue()
)


In [ ]:
# Download from S3
s3_client = boto3.client('s3')
response = s3_client.get_object(Bucket=S3_BUCKET_NAME, Key="loaders/data_loader_01.pth")
model_data = response['Body'].read()

buffer = io.BytesIO(model_data)
loader_3 = torch.load(buffer, weights_only=False)

In [ ]:
for i in loader_3:
    print(i.keys())
    print(i["video_data"].shape)
    print(i["audio_data"].shape)
    print(i["labels"])
    break